#**PROJECT 2: FEATURE MATCHING**

##**1. Background**
In this project, first you will familiarize yourself with OpenCV functions for computing image derivatives and gradients. In particular, you will use the Sobel operators to find edge features in an input image. Then you will write functions for feature matching.

Feature matching is the process of recognizing features of the same object(s) across images of a scene. In this project, we will learn how to extract distinctive key points from an image, how to compute a local feature descriptor from a normalized region around each key point and how to find the corresponding local descriptors of two images of a scene taken from slightly different view points.

You have been given two pairs of images, i.e., NotreDame, and Mount_Rushmore. These images are available under the 'data' folder. You may plot these images and observe that each pair of images corresponds to an object, but taken from different view points.  

### **Your Tasks:**

1.  **Feature/Keypoint Detection:**   Finds interest points in the input images (graded activity). Implement the Harris corner detector (Szeleski 7.1.1).
2.   **Feature Description:** Each region around detected keypoint locations is converted into a more compact and stable (invariant) descriptor that can be matched against other descriptors (graded activity). We will use Scale Invariant Feature Transform (SIFT)-like descriptor (Szeleski 7.1.2).
3.   **Feature Matching:** Finds matching features in multiple images(graded activity). Implement ratio-test (nearest neighbor distance ratio) method for matching features (Szeleski 7.1.3, Equation 7.18)
4.   Finally visualizes the matches

Note that we will use some functions from `skimage` library (https://scikit-image.org/) along with `OpenCV` functions. scikit-image (`skimage`) is a collection of algorithms for image processing written in Python.

Before executing the following python code, you have to upload the `Feature_Matching` folder to your Google drive. Download the project from github repo and upload it into the `My Drive/Colab Notebooks` of your google drive. Now, let us mount the google drive and set the working directory to `My Drive/Colab Notebooks/Fearure_Matching`.

In [ ]:
import os
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
root_dir = "/content/gdrive/MyDrive/"
project_folder = "Colab Notebooks/Feature_Match/"
os.chdir(root_dir+project_folder)

#**2. Gradient Operators and Canny Edge detector(20%)**

Before you start with the feature matching, we ask you to familiarize yourself with the Sobel operators that we discussed in the lecture. OpenCV provides function [Sobel()](https://docs.opencv.org/3.4/d4/d86/group__imgproc__filter.html#gacea54f142e81b6758cb6f375ce782c8d) to calculate the derivatives from an image. The syntax follows.

`dst = cv.Sobel(src, ddepth, dx, dy, ksize, scale, delta, borderType)`

Parameters:



*  **src**	input image.
*   **dst**	output image of the same size and the same number of channels as src .
* **ddepth**	output image depth, see [combinations](https://docs.opencv.org/3.4/d4/d86/group__imgproc__filter.html#filter_depths); in the case of 8-bit input images it will result in truncated derivatives.
* **dx**	order of the derivative x (vertical edges).
*  **dy**  	order of the derivative y (hrizontal edges).
* **ksize**	size of the extended Sobel kernel; it must be 1, 3, 5, or 7.
* **scale**	[optional] scale factor for the computed derivative values; by default, no scaling is applied (see getDerivKernels for details).
* **delta**	[optional] delta value that is added to the results prior to storing them in dst.
* **borderType**	pixel extrapolation method, see BorderTypes. BORDER_WRAP is not supported.


In the following code, we load the `lenna` image, convert it to a gray scale image for ease of computation (need to handle only one channel), and smooths that image using the Gaussian kernel. Then we apply the sobel() function on the `lenna` image to get the vertical and horizontal derivatives. Further, we use a weighted combination of these derivatives values as the final image gradients. 

Your task is to compute the gradient magnitude and oprientations and display the corresponding images. To compute the actual image gradients and orientations, you can use the gradiant magnitude formula from the lecture slide. 




In [ ]:
#Graded activity 1

import cv2 as cv
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import numpy as np
from skimage import io, filters, feature

#Read the image
lenna = cv.imread("./data/lenna.bmp", 1)

#Convert it to a gray scale aimge
gray = cv.cvtColor(lenna, cv.COLOR_BGR2GRAY)

#Remove noise
gray_smooth = cv.GaussianBlur(gray, (3, 3), 0)

#Sobel Operator (horizontal)
grad_x = cv.Sobel(gray_smooth, cv.CV_32F, 1, 0, ksize=3, borderType=cv.BORDER_DEFAULT)

#Sobel Operator (vertical)
grad_y = cv.Sobel(gray_smooth, cv.CV_32F, 0, 1, ksize=3, borderType=cv.BORDER_DEFAULT)

#Convert to uint8
abs_grad_x = cv.convertScaleAbs(grad_x)
abs_grad_y = cv.convertScaleAbs(grad_y)    
    
grad = cv.addWeighted(abs_grad_x, 0.5, abs_grad_y, 0.5, 0)

cv2_imshow(grad)

# Compute magnitude and orientations

#compute the squares, cv.multiply()
sobelx2 = _________
sobely2 = _________

#compute the square root of the sum, check cv.sqrt()
magnitude = _________

cv2_imshow(magnitude)


#compute the orientation, you may use cv.phase()
orientation = __________

orientation = orientation / 2. 
hsv = np.zeros_like(lenna)
hsv[..., 0] = orientation # H (in OpenCV between 0:180)
hsv[..., 1] = 255 # S
hsv[..., 2] = cv.normalize(magnitude, None, 0, 255, cv.NORM_MINMAX) # V 0:255


bgr = cv.cvtColor(hsv, cv.COLOR_HSV2BGR)
cv2_imshow(bgr)



Next task for you is to try different gradient operators available in OpenCV. Try Prewitt, Roberts and Canny edge detectors on `smu` image. Ry varying the hysteresis thresholds of Canny detector and analyze the results.

In [ ]:
#Graded activity 2
smu = cv.imread("./data/smu.jpg", 1)
gray= cv.cvtColor(smu, cv.COLOR_BGR2GRAY)

smu_gray = cv.GaussianBlur(gray, (3, 3), 0)

#Roberts kernel, refer to slides for the v and h kernels
roberts_v = np.array(_______ )  
roberts_h = np.array(_______ )

#Convolve with Roberts kernels
r_grad_x = cv.filter2D(smu_gray, cv.CV_32F, roberts_v)
r_grad_y = cv.filter2D(smu_gray, cv.CV_32F, roberts_h)

robertsx2 = ________
robertsy2 = ________
r_magnitude = _______

cv2_imshow(r_magnitude)

#Prewitt kernels, see the lecture slides
p_kernelx = np.array(______)
p_kernely = np.array(______)


#Convolve with Prewitt kernels
p_grad_x = cv.filter2D(smu_gray, cv.CV_32F, p_kernelx)
p_grad_y = cv.filter2D(smu_gray, cv.CV_32F, p_kernely)

prewittx2 = _______
prewitty2 = _______

p_magnitude = _______

cv2_imshow(p_magnitude)


#Canny edge detector, see cv.Canny()
smu_canny = cv.Canny(smu, 80, 150)
cv2_imshow(smu_canny)



# **3. Load and Display images**

Test data is available under the `data` folder. Read a pair of test images. Below I have used the images of famous `NotreDame`, Paris.

Load and plot the input images.  When you plot the images, you can see that the images are of NotreDame taken from slightly different view points. Your final objective is to map the corresponding features in both the images. For instance, top left part of the buildin in image 1 should be mapped to the same part of the building in image 2.

In [ ]:
file_name = "NotreDame"
file1 = "./data/NotreDame/"+file_name+"1.jpg"
file2 = "./data/NotreDame/"+file_name+"2.jpg"

#Read color image
image1 = cv.imread(file1, 1)
image2 = cv.imread(file2, 1)


#Plot the images
cv2_imshow(image1)
cv2_imshow(image2)

In [ ]:
import numpy as np

img1 = cv.cvtColor(image1, cv.COLOR_BGR2GRAY)
img2 = cv.cvtColor(image2, cv.COLOR_BGR2GRAY)


#Resacle the images to their half sizes (bilinear interpolation)[To speed up the computation]

img1 = cv.resize(img1, None, fx=0.5, fy=0.5, interpolation = cv.INTER_LINEAR)
img2 = cv.resize(img2, None, fx=0.5, fy=0.5, interpolation = cv.INTER_LINEAR)

cv2_imshow(img1)
cv2_imshow(img2)

##**3. Key Points Extraction (30%)**
The next task is to extract distinctive key points (or interest points). There are many different ways to find key points in an image. We can use Harris corners detection (to be discussed in the next lecture), largely due to its simplicity. 

The Harris corners detection algorithm is based on a somewhat intuitive fact: image intensities adjacent to an object corner are generally dissimilar to the intensities at the corner. A method to find corners is to compute image gradients and determine the image locations with large changes in all the directions. The greater the gradient, the more likely a particular point corresponds to an object corner.

To the right are the (magnified) results of my own Harris corners detector, run on the left image of the Notre Dame cathedral shown above. The detector tends to fire at corners and edges of the building, and in the center, at the paned round glass window.

We first make images smaller to speed up the algorithm. This parameter gets passed into the evaluation code, so don't resize the images except for changing this parameter - We will evaluate your code using scale_factor = 0.5, so be aware of this

Set the pixel width and pixel height of each local feature

In [ ]:
feature_width = 16
   

 Find distinctive points in each image. See Szeliski 4.1.1.
 You have to implement extract_key_points () function. 

 Implement the Harris corner detector (See Szeliski 4.1.1) to start with.     You do not need to worry about scale invariance or keypoint orientation stimation for your Harris corner detector. If you're finding spurious interest point detections near the boundaries, it is safe to simply suppress the gradients / corners near the edges of  the image. 
 
 **Useful functions:** A working solution does not require the use of all of these     functions, but depending on your implementation, you may find some useful. Please  reference the documentation for each function/library and feel free to come to office hours
   
        - skimage.feature.peak_local_max (experiment with different min_distance values to get good results)
        - skimage.measure.regionprops

  
 The following is a graded function.

In [ ]:
#Graded Activity 3
def extract_key_points(image, feature_width):
    '''
    Returns key points for the input image

    :params:
    :image: a grayscale 
    :feature_width:

    :returns:
    :xs: an np array of the x coordinates of the interest points in the image
    :ys: an np array of the y coordinates of the interest points in the image 
   
    '''
    alpha = 0.06
    threshold = 0.01
    stride = 2
    sigma = 0.1
    min_distance = 3   

    
    #step1: blur image. Use filters.gaussian with sigma=0.1, refer to https://scikit-image.org/docs/stable/api/skimage.filters.html#skimage.filters.gaussian
    filtered_image = filters.gaussian(image, sigma=0.1)
    
    # step2: calculate gradient of image, I_x and I_y. Use filters.sobel_v and filters.sobel_h operators
    I_x = _____________
    I_y = _____________

    # step3: calculate I_xx, I_xy, I_yy. Useful functions: np.square() and np.multiply(). Refer to https://numpy.org/doc/stable/reference/routines.math.html
    I_xx = ____________
    I_xy = ____________
    I_yy = ____________

    I_xx = filters.gaussian(I_xx, sigma=0.1)
    I_xy = filters.gaussian(I_xy, sigma=0.1)
    I_yy = filters.gaussian(I_yy, sigma=0.1)

    listC = np.zeros_like(image)

    # step4: caculate C matrix
    for y in range(0, image.shape[0]-feature_width, stride):
        for x in range(0, image.shape[1]-feature_width, stride):
          
            # Matrix
            Sxx = np.sum(I_xx[y:y+feature_width+1, x:x+feature_width+1])
            Syy = np.sum(I_yy[y:y+feature_width+1, x:x+feature_width+1])
            Sxy = np.sum(I_xy[y:y+feature_width+1, x:x+feature_width+1])

            #COmpute the determinant of C
            detC = ______________

            #Compute the trace of C
            traceC = ______________

            C = detC - alpha*(traceC**2)
            
            if C > threshold:
                listC[y+feature_width//2, x+feature_width//2] = C

    # step5: using non-maximal suppression
    ret = feature.peak_local_max(listC, min_distance=min_distance, threshold_abs=threshold)
    return ret[:, 1], ret[:, 0]

Now let us use the function to extract the key points

In [ ]:
print("Key Points Extraction...")
(x1, y1) = extract_key_points(img1, feature_width)
(x2, y2) = extract_key_points(img2, feature_width)

    

Visualize the extracted key points. Use `plt.imshow()` to plot the images. Refer to https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.imshow.html for the parameter details of `plt.imshow()`.
  

In [ ]:
plt.imshow(img1, cmap="gray")
plt.scatter(x1, y1, alpha=1, s=2)
plt.show()

plt.imshow(img2, cmap="gray")
plt.scatter(x2, y2, alpha=1, s=2)
plt.show()

print("Key Points Extraction Completed!")

# **4. Computing the Feature Descriptor (SIFT)(40%)**
Description will be available next week

# **5. Feature Matching (10%)**

 Dsecription will be available next week

#**6. Evaluation Details**
This tutorial project is meant to be done in three recitations (Sept 28, Oct. 5 and Oct 12). You will get a total of 16 days to complete the activities. Attendance in each recitation is mandatory to get marks for the graded activities. We will record your attendance. If you complete all the activities in the first recitation itself, then the second recitation is optional. You have to read the instructions, refer to the slides and do all the exercises, refer to the materials as and when necessary (especially for syntaxes). Evaluation will be done in the recitations, mainly in the second recitation. The marker will come evaluate your work during the recitation on Oct. 12. During the evaluation, you will be asked to show the graded activities and you may expect a couple of related questions.

**NOTE** In case you are unable to attend the recitation, you have to inform the instructor before the recitation and absence due to sickness orr other genuine grounds will be considered.

#**7. References/Source**
1. James Hayes, Georgia Tech, CS 4476-B / 6476-A Computer Vision Course